<H2 style="color:blue;font-size:20px;"align="right">فاطمه شاه حسینی</H2>
 
 <H2 style="color:blue;font-size:20px;"align="right"> 810199440 </H2>
 <H2 style="color:blue;font-size:20px;"align="right"> (Decision Trees, Classification, Linear Regression, Ensemble Learning)  تمرین کامپیوتری شماره چهار </H2>
 <H2 style="color:green;font-size:20px;"align="right"> هدف </H2>
 <H2 style="color:black;font-size:15px;"align="right"> آشنایی با روشهای یادگیری ماشین با کمک کتابخانه های پایتون  </H2>
 <H2 style="color:green;font-size:20px;"align="right"> توضیح کلی پروژه </H2>
 <H2 style="color:black;font-size:15px;"align="right">  این پروژه در  4فاز انجام میشود. در فاز صفر بررسی و تجزیه داده ها انجام میشود و در فاز اول پیش پردازش و رگرسیون خطی انجام میدهیم سپس در فاز دوم
با استفاده از مدلهای طبقه بندی به پیش بینی میپردازیم و در فاز اخر با روشهای یادگیری تجمعی آشنا میشویم </H2>

In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
import numpy as np
import statistics
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import copy
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

In [2]:
ds = pd.read_csv('./house_data.csv')
ds.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,location,style,house_age
0,0,0,0,7129300520,20141013T000000,221900.0,3,1.00,1180.0,5650,...,1955.0,0,98178,47.5112,-122.257,1340,5650,Suburban,Contemporary,67
1,1,1,1,6414100192,20141209T000000,538000.0,3,2.25,2570.0,7242,...,NaN,1991,98125,47.7210,-122.319,1690,7639,Suburban,Ranch,71
2,2,2,2,5631500400,20150225T000000,180000.0,2,1.00,770.0,10000,...,1933.0,0,98028,47.7379,-122.233,2720,8062,Rural,Victorian,89
3,3,3,3,2487200875,20141209T000000,604000.0,4,3.00,1960.0,5000,...,1965.0,0,98136,47.5208,-122.393,1360,5000,Urban,Colonial,57
4,4,4,4,1954400510,20150218T000000,510000.0,3,2.00,1680.0,8080,...,1987.0,0,98074,47.6168,-122.045,1800,7503,Suburban,Cape Cod,35


In [3]:
 pd.Series({c: len(ds[c].unique()) for c in ds})


Unnamed: 0.2     21613
Unnamed: 0.1     21613
Unnamed: 0       21613
id               21436
date               372
price             3625
bedrooms            15
bathrooms           32
sqft_living        967
sqft_lot          9782
floors               7
waterfront           2
view                 5
condition            5
grade               13
sqft_above         946
sqft_basement      305
yr_built           117
yr_renovated        70
zipcode             70
lat               5034
long               752
sqft_living15      777
sqft_lot15        8689
location             3
style                5
house_age          116
dtype: int64

<H2 style="color:green;font-size:20px;"align="right"> ساختار کلی داده </H2>

<link rel="stylesheet" href="fonts/css/fonts.css" />

In [4]:
ds.describe()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,house_age
count,21613.00000,21613.00000,21613.00000,2.161300e+04,2.161300e+04,21613.000000,21613.000000,18528.000000,2.161300e+04,18530.000000,...,21613.000000,21184.000000,18531.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,10806.00000,10806.00000,10806.00000,4.580302e+09,5.401822e+05,3.367464,2.111900,2077.569031,1.510697e+04,1.492903,...,1788.390691,292.121082,1970.896714,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,50.994864
std,6239.28002,6239.28002,6239.28002,2.876566e+09,3.673622e+05,0.943028,0.780697,953.971679,4.142051e+04,0.538562,...,828.090978,442.637777,29.406420,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,29.373411
min,0.00000,0.00000,0.00000,1.000102e+06,7.500000e+04,-5.000000,-5.000000,-33323.000000,5.200000e+02,1.000000,...,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,7.000000
25%,5403.00000,5403.00000,5403.00000,2.123049e+09,3.219500e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,...,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,25.000000
50%,10806.00000,10806.00000,10806.00000,3.904930e+09,4.500000e+05,3.000000,2.250000,1920.000000,7.618000e+03,1.500000,...,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,47.000000
75%,16209.00000,16209.00000,16209.00000,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,...,2210.000000,560.000000,1996.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,71.000000
max,21612.00000,21612.00000,21612.00000,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,...,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,122.000000


In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0.2   21613 non-null  int64  
 1   Unnamed: 0.1   21613 non-null  int64  
 2   Unnamed: 0     21613 non-null  int64  
 3   id             21613 non-null  int64  
 4   date           21613 non-null  object 
 5   price          21613 non-null  float64
 6   bedrooms       21613 non-null  int64  
 7   bathrooms      21613 non-null  float64
 8   sqft_living    18528 non-null  float64
 9   sqft_lot       21613 non-null  int64  
 10  floors         18530 non-null  float64
 11  waterfront     21613 non-null  int64  
 12  view           21613 non-null  int64  
 13  condition      21613 non-null  int64  
 14  grade          21613 non-null  int64  
 15  sqft_above     21613 non-null  int64  
 16  sqft_basement  21184 non-null  float64
 17  yr_built       18531 non-null  float64
 18  yr_ren

<H2 style="color:green;font-size:20px;"align="right">برای هر ویژگی  (missing data) نسبت داده های از دست رفته   </H2>

In [6]:
ds.isnull().sum()/len(ds)

Unnamed: 0.2     0.000000
Unnamed: 0.1     0.000000
Unnamed: 0       0.000000
id               0.000000
date             0.000000
price            0.000000
bedrooms         0.000000
bathrooms        0.000000
sqft_living      0.142738
sqft_lot         0.000000
floors           0.142646
waterfront       0.000000
view             0.000000
condition        0.000000
grade            0.000000
sqft_above       0.000000
sqft_basement    0.019849
yr_built         0.142599
yr_renovated     0.000000
zipcode          0.000000
lat              0.000000
long             0.000000
sqft_living15    0.000000
sqft_lot15       0.000000
location         0.000000
style            0.000000
house_age        0.000000
dtype: float64

<H2 style="color:green;font-size:20px;"align="right"> نمودار وابستگی ویژگی های مختلف به ستون هدف  </H2>

In [8]:
ds.drop(['date'], axis=1)
corr = ds.corr()
corr_with_price = corr['price']
corr

ValueError: could not convert string to float: '20141013T000000'

In [ ]:
ds_without_price = corr_with_price.drop('price')
ds_without_price.plot(kind="bar")

<H2 style="color:black;font-size:15px;"align="right">نسبت به بقیه ویژگی ها، وابستگی بیشتری به قیمت خانه دارند (sqft_lot, sqft_above) ویژگی های    </H2>

<H2 style="color:green;font-size:20px;"align="right"> نمودار تعداد داده های منحصر به فرد در ویژگی های مرحله قبل  </H2>

In [ ]:
plt.figure(figsize=(8,5))
ds.hist(column = "sqft_lot")
plt.show()

In [ ]:
ds.hist(column = "sqft_above")
plt.show()

In [ ]:
ds.hist(column = "sqft_living")
plt.show()

<H2 style="color:green;font-size:20px;"align="right">(scatter plot) بررسی  دقیق تر ارتباط ویژگی ها با ستون قیمت با استفاده از     </H2>

In [ ]:
y = ds['price']

# Loop through the other columns and create a scatter plot for each
for col in ds.columns:
    if col != 'price':
        x = ds[col]
        plt.scatter(x, y)
        plt.ylabel('Price')
        plt.xlabel(col)
        plt.show()

In [ ]:
# # Loop through the other columns and create a scatter plot for each
for col in ds.columns:
    if col != 'price'and col != 'date' and col != 'location' and col != 'style':
        x = ds[col]
        plt.hexbin(x, y)
        plt.ylabel('Price')
        plt.xlabel(col)
        plt.show()

<H2 style="color:green;font-size:20px;"align="right"> روش های حل مشکل داده های از دست رفته  </H2>
<H2 style="color:black;font-size:15px;"align="right"> یک راه حذف ستون با داده های ناقص است. که چون ممکن است ستون ، حاوی ویژگی تاثیرگذاری باشد، حذف آن در ساخت مدل و نتیجه نهایی موثر است و بنابراین این روش 
بیشتر در مواقعی مفید است که تعداد نواقص یک ستون زیاد باشد</H2>
<H2 style="color:black;font-size:15px;"align="right">روش بعدی حذف ردیف با نواقص زیاد است</H2>
<H2 style="color:black;font-size:15px;"align="right"> روش دیگر پر کردن ناقصی های ستون ها با میانگین یا مد هر ستون است. که با وجود اینکه روی طیف داده ها تاثیر منفی ندارد ولی می تواند باعث 
 ایجاد تغییر در پراکندگی و واریانس شود و البته امکان دارد دلیل خالی ماندن بعضی خانه ها این باشد که ویژگی های مخصوص آن خانه اشتراکاتی داشتند و به خاطر همان اشتراک از پر کردن
    خانه امتناع کرده اند و پر کردن با میانگین عملا صحت داده ها را به خطر می اندازد</H2>


<H2 style="color:green;font-size:20px;"align="right"> کدام ویژگی ها بیشترین میزان گم شدن دیتا را دارند؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">  مقادیر گمشده دارند و چون داده ها عددی هستند، پر کردن داده های گمشده با مد ستون راه خوبی است yr_built  و sqft_basement و floors و sqft_living در این دیتاست،  </H2>

In [ ]:
ds = ds.apply(lambda x:x.fillna(x.value_counts().index[0]))
# ds.isnull().sum()
ds['sqft_living']

<H2 style="color:green;font-size:20px;"align="right"> در ویژگی های عددی استاندار کردن و نرمالایز کردن دیتا به چه منظور انجام می شود؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">  استاندارد کردن و نرمالایز کردن دیتا از متداول ترین شیوه های اسکیل کردن هستند که برای یکسان سازی داده ها به کار می روند و 
هدفشان این است که بتوان داده هایی که تفاوت زیادی با هم دارند را در بازه مشخصی قرار داد و با هم مقایسه کرد</H2>
<H2 style="color:black;font-size:15px;"align="right"> نزمالیزیشن داده ها را طوری تغییر می دهد که بین صفر و یک قرار بگیرند. هر داده منهای مینیمم داده ها تقسیم بر مقدار ماکسیمم منهای مینیمم. </H2>
<H2 style="color:black;font-size:15px;"align="right">استانداردیزیشن داده ها را طوری اسکیل می کند که میانگین، صفر و انحراف از معیار،  یک شود. هر داده منهای میانگین تقسیم بر انحراف معیار کل داده ها می شود </H2>
<H2 style="color:black;font-size:15px;"align="right"> اما در شرایطی که توزیع داده ها برای ما مشخص نیست یا می دانیم توزیع به صورت نرمال یا گاوسی نیست بهتر است از این شیوه استفاده کنیم. این روش در الگوریتم هایی که توزیع خاصی برای داده ها در نظر نمی گیرند به کار می آید. مانند  </H2>
<H2 style="color:black;font-size:15px;"align="right">(neural networks و k_nearest neighbors)</H2>

In [ ]:
numeric = ds.select_dtypes(include=['int64', 'float64']).columns
ds[numeric] =(ds[numeric]-ds[numeric].min())/(ds[numeric].max()-ds[numeric].min())

<H2 style="color:green;font-size:20px;"align="right"> در ویژگی های دسته ای استفاده از چه پیش پردازش هایی مفید اند؟ آیا همه داده های دسته ای به این کار نیاز دارند؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">(label encoding, one-hot-encoding)</H2>
<H2 style="color:black;font-size:15px;"align="right">برای ترین کردن داده های غیرعددی و دسته ای، لازم است دسته های مختلف را با اعداد برچسب گذاری کنیم 
و یا با استفاده از نمایش وان هات 
میتوان به تعداد دامنه متغیر بیت تعیین کرد و همه بیت ها به جز بیت مربوط به گزینه موجود در نمونه را صفر قرار داد. همه داده های دسته ای به این تبدیل و کد شدن نیازی ندارند و باید با توجه به نقش هر فیلد جداگانه درباره آن تصمیم گیری کرد </H2>

In [ ]:
ds['location'] = ds['location'].astype('category')
ds['location'] = ds['location'].cat.codes

ds['style'] = ds['style'].astype('category')
ds['style'] = ds['style'].cat.codes

ds.head()

<H2 style="color:green;font-size:20px;"align="right"> امکان حذف بعضی ستون ها وجود دارد؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">بله، مثلا ستون مربوط به ای دی خانه، یا تاریخ هیچ تاثیری بر قیمت خانه ندارند تقریبا منحصر بفرد اند و  برای شناسایی هر خانه استفاده می شوتد</H2>
<H2 style="color:green;font-size:20px;"align="right"> با چه نسبتی و چطور باید داده های تست و ترین را دسته بندی کرد؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">   در اکثر اوقات، بیشتر نمونه ها را به داده ترین اختصاص می دهیم. ولی نسبت مشخصی ندارد و بستگی به تعداد نمونه ها و کلاس ها و مشخصات داده ها دارد. مثلا اگر بخواهیم 1000 داده را با نسبت 80 به 20 به ترین و تست اختصاص دهیم، باید به 20 درصد دیتا را به طور رندوم جدا کنیم برای تست و بقیه را به ترین کردن مدل اختصاص دهیم   </H2>
<H2 style="color:green;font-size:20px;"align="right"> دیتای ولیدیشن چیست؟ </H2>
<H2 style="color:black;font-size:15px;"align="right">  برای جلوگیری از اورفیت شدن ترین، هربار مدل ترین شده را روی بخشی از دیتا که همان دیتای ولیدیشن می باشد، امتحان می کنیم، با توجه به نتایج و دقت مدل، مقدار هایپر پارامتر ها را تغییر می دهیم . جاییکه دقت مدل کردن دیتای ولیدیشن برخلاف دیتای ترین، شروع به کاهش کرد، یعنی از آنجا به بعد ، مدل در حال اورفیت شدن است و باید همانجا لرنینگ را متوقف کرد  </H2>

In [ ]:
ds.sample(frac=1)
n = int(0.8 * len(ds))

train_data = ds[:n]
test_data = ds[n + 1:]

# Linear Regression

Main form of simple linear regression function: 
$$f(x) = \alpha x + \beta$$

here we want to find the intercept($\alpha$) and slope($\beta$) by minimizing the derivation of the RSS function:

- step 1: Compute RSS of the training data  

$$ RSS = \Sigma (y_i - (\hat{\beta} + \hat{\alpha} * x_i) )^2 $$

- step 2: Compute the derivatives of the RSS function in term of $\underline{\alpha}$ and $\underline{\beta}$, and set them equal to 0 to find the desired parameters

$$ \frac{\partial RSS}{\partial \beta} = \Sigma (-f(x_i) + \hat{\beta} + \hat{\alpha} * x_i) = 0$$
$$ \to \hat{\beta} = \hat{y} - \hat{\alpha} \hat{x} \to (1)$$


$$ \frac{\partial RSS}{\partial \alpha} = \Sigma (-2 x_i y_i + 2 \hat{\beta} x_i + 2\hat{\alpha} x_i ^ 2) = 0 \to (2)$$

$$ (1) , (2) \to \hat{\alpha} = \frac{\Sigma{(x_i - \hat{x})(y_i - \hat{y})}}{\Sigma{(x_i - \hat{x})^2}}
$$ 
$$ \hat{\beta} = \hat{y} - \hat{\alpha} \hat{x}$$



Based on the formula above, complete this function to compute the parameters of a simple linear regression

In [ ]:
def simple_linear_regression(input_feature, output):

    # compute the sum of input_feature and output
    x_mean = sum(input_feature) / len(input_feature)
    y_mean = sum(output) / len(output)
    
    # compute the product of the output and the input_feature and its sum
    numerator_sum , denominator_sum = 0, 0
    for i in range(len(input_feature)):
        numerator_sum += ( (input_feature[i] - x_mean ) * (output[i] - y_mean) )
        denominator_sum += (input_feature[i] - x_mean ) * (input_feature[i] - x_mean )
    
    # use the formula for the slope
    slope = numerator_sum / denominator_sum
    
    # use the formula for the intercept
    intercept = y_mean - slope * x_mean
    
    return (intercept, slope)

In [ ]:
best_feature = train_data['sqft_living']
price = train_data['price']
intercept, slope = simple_linear_regression(best_feature, price)
print("alpha = ",slope, ", beta = ", intercept)

<H2 style="color:red;font-size:25px;"align="right"> متد های ارزیابی رگرسیون خطی </H2>
<H2 style="color:green;font-size:20px;"align="right"> Residual Sum of Squares (RSS) </H2>
<H2 style="color:black;font-size:20px;"align="right"> مجموع مربعات باقیمانده ، یک تکنیک آماری است که برای توصیف تفاوت در واریانس مقادیر مشاهده شده در برابر مقادیر پیش بینی شده استفاده می شود. این معیار خوبی است که می تواند برای تجزیه و تحلیل میزان تناسب مجموعه ای از نقاط داده با مدل واقعی مورد استفاده قرار گیرد. </H2>
<H2 style="color:black;font-size:20px;"align="right">RSS = ∑ni=1 (yi - f(xi))2</H2>

<H2 style="color:green;font-size:20px;"align="right"> R2 score (R Square) </H2>
<H2 style="color:black;font-size:20px;"align="right"> مجموع مربعات ، یک تکنیک آماری است که می گوید چه نسبتی از واریانس در متغیر پاسخ،  می تواند توسط متغیر(های) پیش بینی کننده در مدل توضیح داده شود </H2>
<img src="rss.png" width="500" height="600">
<img src="rss2.png" width="300" height="400">

<H2 style="color:green;font-size:20px;"align="right"> Mean Squared Error (MSE) </H2>
<H2 style="color:black;font-size:20px;"align="right"> این روش برآوردگر (روشی برای تخمین کمیت مشاهده نشده)، میانگین مربعات خطاها را اندازه گیری می کند - یعنی میانگین مجذور اختلاف بین مقادیر برآورد شده و آنچه تخمین زده می شود</H2>
<img src="mse.png" width="300" height="400">

<H2 style="color:green;font-size:20px;"align="right"> Root Mean Square Error (RMSE) </H2>
<H2 style="color:black;font-size:20px;"align="right"> این متد جذر واریانس باقیمانده ها را نشان می دهد و به ما ایده ای از فاصله متوسط ​​بین مقادیر داده های مشاهده شده و مقادیر داده های پیش بینی شده می دهد </H2>
<img src="rmse.jpg" width="400" height="500">

Now complete this function to predict the value of given data based on the calculated intercept and slope

In [ ]:
def get_regression_predictions(input_feature, intercept, slope):

    # calculate the predicted values:
    predicted_values = slope * input_feature + intercept

    return (predicted_values)

In [ ]:
train_res = get_regression_predictions(train_data['sqft_living'], intercept, slope)
prices = train_data['price']

for i in range(len(train_data)):
    print("train result: ", train_res[i])
    print("real price:   ", prices.iloc[i])

In [ ]:
test_res = get_regression_predictions(test_data['sqft_living'], intercept, slope)
prices = test_data['price']

for i in range(17291, 21612):
    print("test result: ", test_res[i])
    print("real price:   ",prices[i])

Now that we have a model and can make predictions let's evaluate our model using Root Mean Square Error (RSME). RMSE is the square root of the mean of the squared differences between the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output.

Complete the following function to compute the RSME of a simple linear regression model given the input_feature, output, intercept and slope:

In [ ]:
def get_root_mean_square_error(predicted_values , output):
    # TO DO:

    # Compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - predicted_values

    # square the residuals and add them up
    sum = 0
    for r in residuals:
        sum += r*r

    # find the mean of the above phrase
    MSE = sum / len(output)
   
    # calculate the root
    RMSE = math.sqrt(MSE)

    return(RMSE)

AS you might guessed, the RMSE has no bound and it is not easy to find out the percentage of fitting the model into data with it. instead, we use R2 score. The R2 score is calculated by comparing the sum of the squared differences between the actual and predicted values of the dependent variable to the total sum of squared differences between the actual and mean values of the dependent variable. Matematically, the R2 score formula is shown as follows:

$$R^2 = 1 - \frac{SSres}{SStot} = 1 - \frac{\sum_{i=1}^{n} (y_{i,true} - y_{i,pred})^2}{\sum_{i=1}^{n} (y_{i,true} - \bar{y}_{true})^2} $$

In this step, complete the following function to calculate the R2 score of a given input_feature, output, intercept, and slope:

In [ ]:
def get_r2_score(predicted_values, output):

    # compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - predicted_values
    total_res = output - ( sum(output)/len(output) )
   
    # square the residuals and add them up -> SSres
    SSres = 0
    for r in residuals:
        SSres += r*r

    # compute the SStot
    SStot = 0
    for r in total_res:
        SStot += r*r

    # compute the R2 score value
    R2_score = 1 - SSres / SStot

    return(R2_score)

Now calculate the fitness of the model and explain the outputs

In [ ]:
predicted_values = train_res
output = train_data['price']

RMSE_evaluation = get_root_mean_square_error(predicted_values , output)
R2score_evaluation = get_r2_score(predicted_values, output)
print(RMSE_evaluation)
print(R2score_evaluation)

In [ ]:
designated_feature_list = ['sqft_living' , 'yr_built' , 'grade' , 'zipcode']
output_feature = 'price'

for input_feature in designated_feature_list:
    #  calculate R2 score and RMSE for each given feature
    intercept, slope = simple_linear_regression(train_data[input_feature], train_data[output_feature])
    predicted_values = get_regression_predictions(test_data[input_feature], intercept, slope)
    RMSE_evaluation = get_root_mean_square_error(test_data[input_feature] , test_data[output_feature])
    R2score_evaluation = get_r2_score(predicted_values, test_data[output_feature])
    print(input_feature , ": ")
    print("RMSE_evaluation = ", RMSE_evaluation)
    print("R2score_evaluation = ", R2score_evaluation)
    

<H2 style="color:black;font-size:20px;"align="right"> همانطور که می بینیم، مقدار ریشه میانگین مربع خطا برای ویژگی اول کمتر و درصد نمونه های درست تشخیص داده شده در آردو بیشتر از سایر ویژگی ها شده که این 
نشان می دهد، ویژگی اول که بیشترین کانولوشن با قیمت را داشت، بهترین ویژگی است که می تواند قیمت خانه را با توجه به آن پیش بینی کرد</H2>

<H2 style="color:red;font-size:25px;"align="right"> فاز دو - طبقه بندی </H2>
<H2 style="color:green;font-size:20px;"align="right"> اضافه کردن ستون سطح قیمت به دیتاست  </H2>

In [ ]:
midian_price = ds['price'].median()
prices = []

for i in range(len(ds['price'])):
    if ds['price'][i] > midian_price:
        prices.append("HIGH") 
    else:
        prices.append("LOW")
        
ds['price_level'] = prices
ds = ds.drop(['date', 'price'], axis=1)

In [ ]:
ds.sample(frac=1)

n = int(0.8 * len(ds))

train_data = ds[:n]
test_data = ds[n + 1:]

<H2 style="color:green;font-size:20px;"align="right"> Confusion matrix  </H2>
<H2 style="color:black;font-size:20px;"align="right"> این ماتریس از چهار خانه تشکیل شده، که داده های تست را برحسب صحت تشخیص توسط مدل، طبقه بندی می کند. خانه اول مربوط به تعداد دیتا  های مربوط به کلاس یک است که توسط مدل به درستی تشخیص داده شده اند، خانه دوم دیتاهایی که به اشتباه در کلاس دو قرار داده شده اند. خانه سوم دیتا هایی که به اشتباه در کلاس اول قرار داده شده اند و خانه چهارم دیتاهایی که به درستی در کلاس دو قرار داده شده اند   </H2>
<img src="cmat.png" width="400" height="500">

In [ ]:
def pltConfusionMatrix(y_test, y_pred , str):
    matrix = confusion_matrix(y_test, y_pred)
    clf = ConfusionMatrixDisplay(matrix, display_labels=['type 0', 'type 1'])
    clf.plot()
    clf.ax_.set(title='Confusion Matrix ' + str, 
                xlabel='Predicted Value', 
                ylabel='Actual Value')
    plt.show()

In [ ]:
#decision Tree

X = ds.drop('price_level', axis=1)
y = ds['price_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
classifier = DecisionTreeClassifier(criterion= "gini", max_depth=10, min_samples_leaf=10, min_samples_split = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
pltConfusionMatrix(y_test, y_pred, 'test')

<H2 style="color:green;font-size:20px;"align="right"> Grid Search  </H2>
<H2 style="color:black;font-size:20px;"align="right"> گرید سرچ تابعی است که با گرفتن داده ترین ، بهترین هایپرپارامتر ها را برای لرنینگ انتخابت می کند تا دچار اورفیت و اندرفیت نشویم و نهایتا به بهبود پرفرمنس منجر می شود </H2>

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [1,5,10],
    'min_samples_split': [2,6,9],
    'min_samples_leaf': [2,5,8,10],
    'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [ ]:
tree.plot_tree(classifier)

In [ ]:
#KNN classifier

# separate the features and outcome
X = ds.drop(['price_level'], axis=1) # all columns except 'outcome'
y = ds['price_level'] # only the 'outcome' column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =0)#74

# initialize and fit the k-nearest neighbors model
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(X_train, y_train)

# make predictions on the test set
y_pred = knn.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
  
print("Accuracy: {:.2f}%".format(accuracy*100))   
pltConfusionMatrix(y_test, y_pred, 'test')

In [ ]:
#Logistic Regression model

X = ds.drop('price_level', axis=1)
y = ds['price_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)#52, 82,0

# create a logistic regression model
clf = LogisticRegression(max_iter=1000)

# fit the model to the training data
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# evaluate the model on the test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))
pltConfusionMatrix(y_test, y_pred, 'test')

<H2 style="color:green;font-size:20px;"align="right"> Overfit and Underfit  </H2>
<H2 style="color:black;font-size:20px;"align="right">  آندرفیت وقتی رخ می دهد که مدل به خوبی ترین نشده و تعداد داده های ترین کم بوده بنابراین مدل ساخته شده،
نه روی داده ترین و نه تست به خوبی کار نمی کند. در این صورت می گوییم داده ها بایاس زیاد و واریانس کم دارند</H2>
<H2 style="color:black;font-size:20px;"align="right">    اور فیت وقتی رخ میدهد که مدل کاملا روی داده ترین ، اموزش دیده و دقیقا خود را با آن (حتی با نویز های آن) منطبق کرده است. 
بنابراین روی داده ترین با دقت خیلی زیاد کار می کند ولی روی داده ی تست دقت بسیار کمتری دارد. به طوریکه این تفاوت دقت در دو داده قابل توجه است.
در این صورت می گوییم داده ها واریانس زیاد و بایاس کم دارند و مدل قابلیت جنریزیشن خود را از دست داده است</H2>

In [ ]:
y_train_pred = clf.predict(X_train)
accuracy = accuracy_score(y_train, y_train_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

<H2 style="color:black;font-size:20px;"align="right"> از آنجایی که مدل ، داده ترین را هم به همان دقت داده تست پیش بینی کرده است، یعنی 
هایپرپارامتر ها به درستی انتخاب شده اند و مدل دچار اورفیت نشده است. همچنین چون مدل به پیش بینی های قابل قبولی داشته است و دقت 89 درصد داشته یعنی دچار اندرفیت هم نشده است</H2>

<H2 style="color:green;font-size:25px;"align="right"> تغییر در پیش پردازش ها   </H2>

In [ ]:
#KNN classifier (Change the number of neighbors from 15 to 1000)

# separate the features and outcome
X = ds.drop(['price_level'], axis=1) 
y = ds['price_level'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =0)#74

# initialize and fit the k-nearest neighbors model
knn = KNeighborsClassifier(n_neighbors=1000)
knn.fit(X_train, y_train)

# make predictions on the test set
y_pred = knn.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
  
print("Accuracy: {:.2f}%".format(accuracy*100))   
pltConfusionMatrix(y_test, y_pred, 'test')

In [ ]:
#decision Tree

X = ds.drop('price_level', axis=1)
y = ds['price_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
classifier = DecisionTreeClassifier(criterion= "gini", max_depth=2, min_samples_leaf=10, min_samples_split = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
pltConfusionMatrix(y_test, y_pred, 'test')

<H2 style="color:balck;font-size:20px;"align="right">همانطور که می بینیم، افزایش تعداد همسایگان در الگوریتم کا ان ان، باعث افت دقت می شود و در الگوریتم درخت جست و جو با کاهش عمق درخت از 10 به 2، مدل دچار اندرفیت شده و دقت آن کاهش می یابد . همچنین با افزایش عمق درخت از 10 به 20، دچار اورفیت شده و دقت مجددا کاهش می یابد.</H2>

<H2 style="color:red;font-size:20px;"align="right"> Ensemble Learning </H2>
<H2 style="color:green;font-size:15px;"align="right">  Random Forest </H2>

In [ ]:
def RandomForest(newFilmData, n, m, target):
    X_train, X_test, y_train, y_test = train_test_split(newFilmData, target, test_size=0.3, random_state=42)
    clf = RandomForestClassifier(n_estimators = n, min_samples_leaf = m).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_trainPred = clf.predict(X_train)
    trainList = accuracy_score(y_test, y_pred)
    testList = accuracy_score(y_train, y_trainPred)
    return trainList, testList

<H2 style="color:green;font-size:20px;"align="right">  بررسی هایپرپارامتر ها و تاثیر آنها بر پیش بینی رندوم فارست </H2>

<H2 style="color:black;font-size:20px;"align="left">n_estimators: Number of trees in the forest.</H2>
<H2 style="color:black;font-size:20px;"align="left">criterion: The function to measure the quality of a split.</H2>
<H2 style="color:black;font-size:20px;"align="left">max_depth: Maximum depth of the tree.</H2>
<H2 style="color:black;font-size:20px;"align="left">min_samples_split: Minimum number of samples required to split an internal node.</H2>
<H2 style="color:black;font-size:20px;"align="left">min_samples_leaf: Minimum number of samples required to be at a leaf node.</H2>
<H2 style="color:black;font-size:20px;"align="left">max_features: Number of features to consider when looking for the best split.</H2>
<H2 style="color:black;font-size:20px;"align="left">max_leaf_nodes: Maximum number of leaf nodes.</H2>
<H2 style="color:black;font-size:20px;"align="left">min_impurity_decrease: A node will be split if this split induces a decrease of the impurity greater than or equal to this value.</H2>
<H2 style="color:black;font-size:20px;"align="left">bootstrap: Whether or not to sample data with replacement.</H2>
<H2 style="color:black;font-size:20px;"align="left">n_jobs: Number of jobs to run in parallel. </H2>

In [ ]:
X = ds.drop(['price_level'], axis=1) 
y = ds['price_level'] 

n_estimators = [10, 50, 100, 150, 200]
max_depth = [1, 5, 10, 15, 20, 25, 30]

# Create a 2D array to store the accuracy for each combination of n_estimators and max_depth
scores = np.zeros((len(n_estimators), len(max_depth)))
for i, n in enumerate(n_estimators):
    for j, d in enumerate(max_depth):
        # Train the Random Forest model with the current combination of n_estimators and max_depth
        clf = RandomForestClassifier(n_estimators=n, max_depth=d)
        clf.fit(X_train, y_train)
        # Calculate the accuracy for the current combination of n_estimators and max_depth
        accuracy = clf.score(X_test, y_test)
        scores[i, j] = accuracy
        print(i, j , accuracy)

# Plot the results as a heatmap
plt.imshow(scores, aspect='auto', origin='lower')
plt.xticks(np.arange(len(max_depth)), max_depth)
plt.yticks(np.arange(len(n_estimators)), n_estimators)
plt.xlabel('Max Depth')
plt.ylabel('Number of Estimators')
plt.colorbar()
plt.show()

<H2 style="color:black;font-size:15px;"align="right"> n_estimators  </H2>
<H2 style="color:black;font-size:15px;"align="right">  تعداد درختانی که با ویژگی های متفاوت ترین شده و درنهایت بین پیش بینی های آنها رای گیری می شود  </H2>
<H2 style="color:black;font-size:15px;"align="right"> max_depth  </H2>
<H2 style="color:black;font-size:15px;"align="right">  بیشترین عمق درخت های تصمیم  </H2>
<H2 style="color:black;font-size:15px;"align="right">همانطور که درنمودار مشخص است با افزایش عمق درخت ها دقت بالاتر می رود و افزایش تعداد درخت ها تا جایی دقت را افزایش می دهد ولی پس از آن کاهش می یابد  ،  </H2>
<H2 style="color:red;font-size:20px;"align="right"> مقایسه رندوم فارست و درخت تصمیم  </H2>
<H2 style="color:black;font-size:20px;"align="right">رندوم فارست نتیجه دقیق تری نسبت به درخت تصمیم ساده می دهد. در درخت تصمیم واریانس بالا و بایاس کم است. مخصوصا وقتی عمق درخت را خیلی زیاد کنیم ، مدل تمام جزییات دیتای ترین را یاد میگیرد برای همین بایاس (که نشان دهنده اختلاف پیش بینی و واقعیت نمونه است) کم می شود که منجر میشود خاصیت جنرالیزیشن خود را از دست داده و  دچار اورفیت شود.
     به طور برعکس در رندوم فارست، چون بین جوابهای درخت های متعدد رای گیری میشود ، واریانس کم و چون به جزییات نتیجه هر درخت پرداخته نمی شود، بایاس زیاد می شود. اینجا خطری که مدل را تهدید می کند اندرفیت شدن است. ولی درکل همانطور که در نتایج می بینیم،  رندوم فارست پیش بینی دقیق تری دارد .</H2>

<H2 style="color:red;font-size:25px;"align="right">Gradiant Boosting </H2>
<H2 style="color:balck;font-size:20px;"align="right">یکی از روش های تقویت کردن دقت یک مدل پیش بینی الگوریتم "افزایش گرادیان" است.این الگوریتم چندین یادگیرنده ضعیف را با هم ترکیب و به یادگیرندگان قویتر تبدیل می کند، به این صورت که در آن هر مدل جدید برای به حداقل رساندن تابع ضرر مدل قبلی با استفاده از نزول گرادیان (گرادینت دیسنت)آموزش داده می شود. در هر تکرار، الگوریتم گرادیان تابع ضرر را با توجه به پیش‌بینی‌های مجموعه فعلی محاسبه می‌کند و سپس یک مدل ضعیف جدید را برای به حداقل رساندن این گرادیان آموزش می‌دهد. سپس پیش‌بینی‌های مدل جدید به مجموعه اضافه می‌شود و این فرآیند تا زمانی که یک معیار توقف برآورده شود، تکرار می‌شود </H2>
<img src="gb.png" width="500" height="600">

<H2 style="color:green;font-size:25px;"align="right"> تفاوت گرادیان بوستینگ و رندوم فارست</H2>
<H2 style="color:balck;font-size:20px;"align="right">بوستینگ مبتنی بر لرنر های ضعیفی مثل درختان کم عمق است که  بایاس زیاد و واریانس کم دارند. دراین روش تقویت خطا عمدتاً با با تجمیع خروجی مدل ها و با کاهش دادن بایاس انجام میشود
از سوی دیگر، رندوم فارست از درختان تصمیم گیری کاملاً رشد یافته استفاده می کند که بایاس کم و واریانس بالا دارند. سپس با کاهش واریانس،  کاهش خطا را انجام میدهد. الگوریتم نمی تواند بایاس را کاهش دهد، بنابراین در ابتدا نیاز به درختان بزرگ و هرس نشده است، به طوری که در ابتدا بایاس تا حد ممکن کم باشد. </H2>
<H2 style="color:green;font-size:25px;"align="right">XGboost</H2>
<H2 style="color:balck;font-size:20px;"align="right"> یک نوع پیاده سازی درخت تصمیم گرادیانت بوستینگ است که تابع ضرر را به حداقل می رساند. دراین پیاده سازی آموزش به صورت مکرر ادامه می‌یابد و درخت‌های جدیدی اضافه می‌شوند که باقی‌مانده یا خطاهای درخت‌های قبلی را پیش‌بینی می‌کنند و سپس با درخت‌های قبلی ترکیب می‌شوند تا پیش‌بینی نهایی را انجام دهند.  از یک الگوریتم نزولی گرادیان برای به حداقل رساندن لاست ها هنگام افزودن مدل‌های جدید استفاده می‌کند و به تدریج شرایط  بیشتری را به درخت تصمیم اضافه می کند تا یک مدل قوی تر بسازد </H2>

In [ ]:
X = ds.drop('price_level', axis=1)
y = ds['price_level']
y = [(1 if i=="HIGH" else 0) for i in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

clf=XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=500, objective='binary:logistic', booster='gbtree')
 
#Creating the model on Training Data
XGB = clf.fit(X_train,y_train)
pred = XGB.predict(X_test)

# evaluate the model's accuracy
accuracy = accuracy_score(y_test, pred)
print('Accuracy:', accuracy)
pltConfusionMatrix(y_test, pred, 'test')

In [ ]:
param_grid = {
    'max_depth': [1,3,5,10,50]
}
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

<H2 style="color:balck;font-size:20px;"align="right">  همانطور که می بینیم با استفاده از الگوریتم گرادیان بوستینگ دقت پیش بینی افزایش یافته است. همچنین از تابع گرید سرچ برای پیدا کردن هایپرپارامتر مربوط به عمق ماکسیمم استفاده کرده ایم </H2>

<H2 style="color:green;font-size:20px;"align="right">منابع </H2>
<H2 style="color:black;font-size:15px;"align="right"> https://www.bing.com/search?EID=MBSC&form=BGGCDF&pc=U710&q=which+version+of+scikitlearn+i+have+to+install%3F <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://scikit-learn.org/stable/modules/tree.html <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://medium.datadriveninvestor.com/an-introduction-to-grid-search-ff57adcc0998 <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://www.geeksforgeeks.org/ml-gradient-boosting/ <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://stats.stackexchange.com/questions/173390/gradient-boosting-tree-vs-random-forest#:~:text=Boosting%20is%20based%20on%20weak%20learners%20%28high%20bias%2C,variance%2C%20by%20aggregating%20the%20output%20from%20many%20models%29. <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://www.geeksforgeeks.org/understanding-logistic-regression/ <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://www.geeksforgeeks.org/xgboost/ <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://www.datacamp.com/tutorial/xgboost-in-python <H2>
<H2 style="color:black;font-size:15px;"align="right"> https://scikit-learn.org/stable/modules/preprocessing.html <H2>
    